# Group 11: Iterative Prisoner's Dilemma

## Description

The [Prisoner's Dilemma](https://en.wikipedia.org/wiki/Prisoner%27s_dilemma) (PD) is a classical game analyzed in game theory, which is widely used to (attempt to) model social/economical interaction. It's a "dilemma" as, if exploited to explain the emergence of altruism in human or in general animal society, it fails badly at a first glance.

The classical situation-representation of the PD is that of two prisoners whose conviction depends on their mutual cooperation. It is easier understood though if illustrated in terms of a trade-off game (closed bag exachange):

*Two people meet and exchange closed bags, with the understanding that one of them contains money, and the other contains a purchase. Either player can choose to honor the deal by putting into his or her bag what he or she agreed, or he or she can defect by handing over an empty bag.* 

![prisioners_example](./dilemma-prisoners.jpg)



It is obvious that for both players the winning strategy is to NOT cooperate.

Things changes when the interaction between the two individuals is iterated, in that case a more altruist attitude (strategy) is expected to emerge. The goal of this project is to test this hypothesis.

Mathematically the PD can be expressed with very basic linear algebra. The key component is the **Payoff matrix** $M$, which quantify the reward each player gets depending on whether she cooperated or not (defect):

$$
M = 
\begin{pmatrix} 
R & S \\
T & P 
\end{pmatrix}
$$

with $T,R,S,P$ integers that satisfy the following conditions:

$$
T>R>P>S; \quad 2R > T+S
$$

for example $T=3$, $R=2$, $P=1$ and $S=0$, or  $T=5$, $R=3$, $P=2$, $S=0$. Each player choice (move) can be represented by one of the two axis in ${\rm I\!R}^2$, i.e. $u_C=\begin{pmatrix} 1 \\ 0 \end{pmatrix}$ or $u_D=\begin{pmatrix} 0 \\ 1 \end{pmatrix}$, where the first coordinate stands for *Cooperate* and the second for *Defect*. Being $u_1$ and $u_2$ their rewards $r_1$ and $r_2$ can be computed then as:

$$
r_1 = u_1^T M u_2
\quad
\quad
r_2 = u_2^T M u_1
$$

In an Iterative Prisoner's Dilemma (IPD), two players play prisoner's dilemma more than once in succession and they remember previous actions of their opponent and change their strategy accordingly. The winning strategy is the one which yields to a larger reward at the end of the IPD.

The strategy can be represented as a function which outputs either $u_C$ or $u_D$. Such function can depend on the opponent's history of moves, her on history of moves, on the number of moves played till that moment and so on, but it can only be based on a probability density function. Used strategies are:

* **Nice guy**: always cooperate (the function's output is always $u_D$)
* **Bad guy**: always defect 
* **Mainly nice**: randomly defect $25\%$ of the times and cooperate $75\%$, $k<50$
* **Mainly bad**: randomly defect $75\%$ of the times and cooperate $25\%$
* **tit-for-tat**: start by cooperating, then repeat what the opponent has done in the previous move 
* **random-guy**: randomly decides their move
* **smart-guy**: starts cooperating, then checks the opponent's history moves, if they cooperate at least 70% of the time, you cooperate, otherwise you defect

Here is how the code of each strategy looks like:

In [1]:
import random
import numpy as np
import re

cooperate = [[1], [0]]
defect = [[0], [1]]

def roll():
    return random.randint(1,8)
  
def always_cooperate(opponentMove = 'any'): #100% cooperates
    return cooperate

def always_defect(opponentMove = 'any'): #100% defects
    return defect

def mainly_nice(opponentMove = 'any'): #75% randomly cooperates
    if roll() > 2:
        return cooperate
    else:
        return defect

def mainly_bad(opponentMove = 'any'): #75% randomly defects
    if roll() > 2:
        return cooperate
    else:
        return defect
    
def tit_for_tat(opponentMove): #starts cooperating, and then mimic opponent's last move
    if opponentMove == 'start':
        return cooperate
    opponentHistory = []
    opponentHistory.append(opponentMove)
    if opponentHistory:
        return opponentHistory[-1]
    else:
        return cooperate
    
def random_guy(opponentMove = 'any'): #randomly decides their move
    if roll() % 2 == 0:
        return defect
    else:
        return cooperate
    
def smart_guy(opponentMove): #checks the opponent's history moves, if they cooperate at least 70% of the time, you cooperate, otherwise you defect
    if opponentMove == 'start':
        return cooperate
    opponentHistory = []
    opponentHistory.append(opponentMove)
    average = sum([[1], [0]] == x for x in opponentHistory)/len(opponentHistory)
    if average > 0.7:
        return cooperate
    else:
        return defect

Now let's define some variables to perform the simulations. 

possible_strategies: a list of functions that we are going to iterate to simulate the games.

test_strategies: a dict that maps integer numbers to different strategies, will be used to the user choose a strategy to test. 

payoff_matrix: the weights that each combined actions will have, coop-coop, defect-coop, coop-defect, defect-defect

In [2]:
possible_strategies = [always_cooperate,always_defect,mainly_nice,mainly_bad,tit_for_tat,random_guy,smart_guy]
test_strategies = {1:always_cooperate,2:always_defect,3:mainly_nice,4:mainly_bad,5:tit_for_tat,6:random_guy,7:smart_guy}
#payoff_matrix = [[3,1],[4,2]]
#payoff_matrix = [[2,0],[3,1]]
#payoff_matrix = [[3,0],[5,2]]
payoff_matrix = [[3,1],[5,0]] #penalyzes defecting

scores = {}
for s in possible_strategies:
  scores[s.__name__] = 0

Now we have to define some functions to actually simulate the game. The first one is the play_game, where we receive strategies from both players and the number of rounds they will play. The number of rounds play a crucial role here since some strategies relies on the history of opponent moves. 

In [3]:
def play_game(player1, player2, rounds):
    # Declare global variables
    global scores
    global payoff_matrix

    # Initialize scores for both players
    p1Score = 0
    p2Score = 0

    # Get initial moves from both players
    p1Move = player1('start')
    p2Move = player2('start')

    # Iterate over the specified number of rounds
    for i in range(rounds):
        # Calculate the payoff for player 1 and player 2 based on their moves
        p1Score += np.dot(np.dot(np.transpose(p1Move), payoff_matrix), p2Move)
        p2Score += np.dot(np.dot(np.transpose(p2Move), payoff_matrix), p1Move)
        
        # In case both players are using the same strategy, update the score only once
        if player1.__name__ == player2.__name__:
            scores[player1.__name__] += p1Score
        else:
            # Update scores for both players
            scores[player1.__name__] += p1Score
            scores[player2.__name__] += p2Score
        
        # Store the previous moves
        prevP1Move = p1Move
        prevP2Move = p2Move

        # Determine the next moves for both players based on the previous moves
        p1Move = player1(prevP2Move)
        p2Move = player2(prevP1Move)
    
    # Determine the winner of the game or if it's a draw
    #if p1Score > p2Score:
    #    print(player1.__name__, 'beats', player2.__name__, p1Score, '-', p2Score)
    #elif p2Score > p1Score:
    #    print(player2.__name__, 'beats', player1.__name__, p2Score, '-', p1Score)
    #else:
    #    print('It was a draw', player1.__name__, p1Score, player2.__name__, p2Score)

Next function we will define is test_strategy, used for 1st assignment. This functions receives two parameters, a chosen strategy and the number of games to be simulated against each other strategy.

In [4]:
def test_strategy(strategy, turns):
    # Initialize the number of games played
    games = 0

    # Iterate over all possible strategies
    for s in possible_strategies:
        # Print the name of the current opponent strategy
        print("Current strategy:", s.__name__)

        # Check if the current strategy is not the same as the given strategy
        if s != strategy:
            # Play two games: given strategy vs. current strategy, and vice versa
            play_game(strategy, s, turns)
            play_game(s, strategy, turns)

            # Increment the number of games played by 2
            games += 2

            # Print an empty line for clarity
            print('')

    # Calculate the average score against different opponents
    opScore = scores[strategy.__name__] / games
    print('Your average score against different opponents was', opScore, '\n')

    # Print a message indicating what the average score would be if everyone followed the given strategy
    print('So if everybody was doing it')

    # Reset scores for all strategies
    for s in possible_strategies:
        scores[s.__name__] = 0

    # Play games against the same strategy for the given number of turns
    for i in range(turns):
        play_game(strategy, strategy, turns)

    # Calculate the average score against the same strategy opponents
    selfScore = scores[strategy.__name__] / (turns)
    print('Your average score against the same strategy opponents was', selfScore)

    # Calculate the overall average score
    print('Your overall average score was', (opScore + selfScore) / 2)

Following the assignments list, we defined the test_round_goblin function, that only receives one parameter, the number of rounds each strategy will play among each other in a round-goblin scheme.

In [5]:
def test_round_goblin(turns):
    # Declare global variable for scores
    global scores

    # Iterate over all pairs of strategies
    for i, strategy1 in enumerate(possible_strategies):
        for j, strategy2 in enumerate(possible_strategies):
            # Ensure that each pair of strategies is only tested once
            if i < j:
                # Play two games: strategy1 vs strategy2 and strategy2 vs strategy1
                play_game(strategy1, strategy2, turns)
                play_game(strategy2, strategy1, turns)
                
                # Update scores for both strategies based on the outcomes of the games
                scores[strategy1.__name__] += scores[strategy2.__name__]
                scores[strategy2.__name__] += scores[strategy1.__name__]
                
                # Print an empty line for readability
                print('')

Finally, we define the function iterate_strategies, which aims to compute the round-goblin scheme tournment with the addition of the following rule: now, we add 33% of new players in each iteration, with the most successful strategy of the previous iteration.

In [6]:
def iterate_strategies(iterations, turns):
    # Declare global variables
    global possible_strategies
    global scores

    # Iterate over the specified number of iterations
    for iteration in range(iterations):
        print(f"Iteration {iteration + 1}:")

        # Create a dictionary to store the average scores for each strategy
        avg_scores = {}

        # Test each strategy against all other strategies
        for strategy in possible_strategies:
            # Initialize variables and reset scores
            compute_avg_score = 1
            games = 0
            scores[strategy.__name__] = 0

            # Play games against all other strategies
            for opponent_strategy in possible_strategies:
                play_game(strategy, opponent_strategy, turns)
                play_game(opponent_strategy, strategy, turns)
                games += 2

            # Compute average score for different encounters with each strategy
            while compute_avg_score:
                if strategy.__name__ + str(compute_avg_score - 1) in avg_scores:
                    compute_avg_score += 1
                else:
                    #print("avg_scores do ", strategy.__name__ + str(compute_avg_score - 1) + " = jogos " + str(games))
                    avg_scores[strategy.__name__ + str(compute_avg_score - 1)] = scores[strategy.__name__] / games
                    compute_avg_score = 0 

        # Sort strategies based on average scores
        sorted_strategies = sorted(avg_scores.items(), key=lambda x: x[1][0][0], reverse=True)
        print(sorted_strategies)

        # Determine the top-performing strategy
        top_strategy = re.sub(r'\d+', '', sorted_strategies[0][0])

        # Increase the population of the top-performing strategy
        new_population = int(len(possible_strategies) / 3)  # Increase population by 33% of current population
        possible_strategies = [globals()[top_strategy]] * new_population + possible_strategies[:len(possible_strategies)]
        
        print("New Population:",[p.__name__ for p in possible_strategies])

        # Reset scores for the next iteration
        scores = {}

        # Print the results of the iteration
        for strategy in possible_strategies:
            scores[strategy.__name__] = 0

        print(f"Top strategy in iteration {iteration + 1}: {top_strategy}")
        print("")

Lastly, we give the option to the user test which option they want to play.

In [7]:
# print('Here are your game options')
print('1: To test your AI strategy against all other AI strategies')
print("2: To test all AI's strategies against each other (round-robin)")
print("3: To test all AI's strategies against each other (round-robin) + increasing the population by the best strategies")
choice = int(input())

if choice == 1:
    print('Here are the available strategies:')
    for key, func in test_strategies.items():
        print(f"{key}: {func.__name__}", end="\n")
    num = int(input('Choose a strategy via number: '))
    strategy = test_strategies[num]
    test_strategy(strategy,20)
elif choice == 2:
    test_round_goblin(10)
elif choice == 3:
    iterate_strategies(iterations=5, turns=20)


1: To test your AI strategy against all other AI strategies
2: To test all AI's strategies against each other (round-robin)
3: To test all AI's strategies against each other (round-robin) + increasing the population by the best strategies


 3


Iteration 1:
[('mainly_nice0', array([[564.14285714]])), ('random_guy0', array([[529.85714286]])), ('always_cooperate0', array([[512.14285714]])), ('tit_for_tat0', array([[509.21428571]])), ('mainly_bad0', array([[507.64285714]])), ('always_defect0', array([[487.85714286]])), ('smart_guy0', array([[482.78571429]]))]
New Population: ['mainly_nice', 'mainly_nice', 'always_cooperate', 'always_defect', 'mainly_nice', 'mainly_bad', 'tit_for_tat', 'random_guy', 'smart_guy']
Top strategy in iteration 1: mainly_nice

Iteration 2:
[('random_guy0', array([[559.]])), ('mainly_nice1', array([[558.88888889]])), ('mainly_nice0', array([[557.44444444]])), ('mainly_nice2', array([[538.88888889]])), ('mainly_bad0', array([[535.77777778]])), ('tit_for_tat0', array([[528.88888889]])), ('always_defect0', array([[528.61111111]])), ('smart_guy0', array([[511.38888889]])), ('always_cooperate0', array([[508.22222222]]))]
New Population: ['random_guy', 'random_guy', 'random_guy', 'mainly_nice', 'mainly_nice', 

In case you select 1, you can expect some output like this:<br>
Here are the available strategies:<br>
1: always_cooperate<br>
2: always_defect<br>
3: mainly_nice<br>
4: mainly_bad<br>
5: tit_for_tat<br>
6: random_guy<br>
7: smart_guy<br>
Choose a strategy via number:  7<br>
Current strategy: always_cooperate<br>
It was a draw smart_guy [[60]] always_cooperate [[60]]<br>
It was a draw always_cooperate [[60]] smart_guy [[60]]<br><br>
Current strategy: always_defect<br>
always_defect beats smart_guy [[5]] - [[1]]<br>
always_defect beats smart_guy [[5]] - [[1]]<br><br>
Current strategy: mainly_nice<br>
It was a draw smart_guy [[60]] mainly_nice [[60]]<br>
mainly_nice beats smart_guy [[56]] - [[52]]<br><br>
Current strategy: mainly_bad<br>
mainly_bad beats smart_guy [[26]] - [[22]]<br>
mainly_bad beats smart_guy [[35]] - [[31]]<br><br>
Current strategy: tit_for_tat<br>
It was a draw smart_guy [[60]] tit_for_tat [[60]]<br>
It was a draw tit_for_tat [[60]] smart_guy [[60]]<br><br>
Current strategy: random_guy<br>
It was a draw smart_guy [[42]] random_guy [[42]]<br>
random_guy beats smart_guy [[38]] - [[34]]<br><br>
Current strategy: smart_guy<br>
Your average score against different opponents was [[434.08333333]] <br><br>
So if everybody was doing it<br>
It was a draw smart_guy [[60]] smart_guy [[60]]<br>
It was a draw smart_guy [[60]] smart_guy [[60]]<br>
It was a draw smart_guy [[60]] smart_guy [[60]]<br>
It was a draw smart_guy [[60]] smart_guy [[60]]<br>
It was a draw smart_guy [[60]] smart_guy [[60]]<br>
It was a draw smart_guy [[60]] smart_guy [[60]]<br>
It was a draw smart_guy [[60]] smart_guy [[60]]<br>
It was a draw smart_guy [[60]] smart_guy [[60]]<br>
It was a draw smart_guy [[60]] smart_guy [[60]]<br>
It was a draw smart_guy [[60]] smart_guy [[60]]<br>
It was a draw smart_guy [[60]] smart_guy [[60]]<br>
It was a draw smart_guy [[60]] smart_guy [[60]]<br>
It was a draw smart_guy [[60]] smart_guy [[60]]<br>
It was a draw smart_guy [[60]] smart_guy [[60]]<br>
It was a draw smart_guy [[60]] smart_guy [[60]]<br>
It was a draw smart_guy [[60]] smart_guy [[60]]<br>
It was a draw smart_guy [[60]] smart_guy [[60]]<br>
It was a draw smart_guy [[60]] smart_guy [[60]]<br>
It was a draw smart_guy [[60]] smart_guy [[60]]<br>
It was a draw smart_guy [[60]] smart_guy [[60]]<br>
Your average score against the same strategy opponents was [[630.]]<br>
Your overall average score was [[532.04166667]] [[630.]]
Your overall average score was [[532.04166667]]

In case you select 2, you can expect some output like this:

always_defect beats always_cooperate [[50]] - [[10]]
<br>
always_defect beats always_cooperate [[50]] - [[10]]
<br>
<br>
mainly_nice beats always_cooperate [[34]] - [[26]]
<br>
mainly_nice beats always_cooperate [[32]] - [[28]]
<br>
<br>
mainly_bad beats always_cooperate [[42]] - [[18]]
<br>
mainly_bad beats always_cooperate [[44]] - [[16]]
<br>
<br>
It was a draw always_cooperate [[30]] tit_for_tat [[30]]
<br>
It was a draw tit_for_tat [[30]] always_cooperate [[30]]
<br>
<br>
random_guy beats always_cooperate [[42]] - [[18]]
<br>
random_guy beats always_cooperate [[38]] - [[22]]
<br>
<br>
It was a draw always_cooperate [[30]] smart_guy [[30]]
<br>
It was a draw smart_guy [[30]] always_cooperate [[30]]
<br>
<br>
always_defect beats mainly_nice [[25]] - [[5]]
<br>
always_defect beats mainly_nice [[50]] - [[10]]
<br>
<br>
always_defect beats mainly_bad [[15]] - [[3]]
<br>
always_defect beats mainly_bad [[15]] - [[3]]
<br>
<br>
always_defect beats tit_for_tat [[5]] - [[1]]
<br>
always_defect beats tit_for_tat [[5]] - [[1]]
<br>
<br>
always_defect beats random_guy [[30]] - [[6]]
<br>
always_defect beats random_guy [[25]] - [[5]]
<br>
<br>
always_defect beats smart_guy [[5]] - [[1]]
<br>
always_defect beats smart_guy [[5]] - [[1]]
<br>
<br>
mainly_bad beats mainly_nice [[34]] - [[14]]
<br>
mainly_bad beats mainly_nice [[27]] - [[15]]
<br>
<br>
It was a draw mainly_nice [[30]] tit_for_tat [[30]]
<br>
It was a draw tit_for_tat [[30]] mainly_nice [[30]]
<br>
<br>
random_guy beats mainly_nice [[31]] - [[23]]
<br>
random_guy beats mainly_nice [[29]] - [[13]]
<br>
<br>
It was a draw mainly_nice [[27]] smart_guy [[27]]
<br>
It was a draw smart_guy [[30]] mainly_nice [[30]]
<br>
<br>
It was a draw mainly_bad [[9]] tit_for_tat [[9]]
<br>
mainly_bad beats tit_for_tat [[14]] - [[10]]
<br>
<br>
mainly_bad beats random_guy [[21]] - [[9]]
<br>
mainly_bad beats random_guy [[18]] - [[6]]
<br>
<br>
mainly_bad beats smart_guy [[26]] - [[22]]
<br>
mainly_bad beats smart_guy [[5]] - [[1]]
<br>
<br>
random_guy beats tit_for_tat [[20]] - [[16]]
<br>
random_guy beats tit_for_tat [[32]] - [[28]]
<br>
<br>
It was a draw tit_for_tat [[30]] smart_guy [[30]]
<br>
It was a draw smart_guy [[30]] tit_for_tat [[30]]
<br>
<br>
It was a draw random_guy [[30]] smart_guy [[30]]
<br>
random_guy beats smart_guy [[32]] - [[28]]

In case you select 3, you can expect some output like this:

Iteration 1:
[('mainly_nice0', array([[521.14285714]])), <br>('mainly_bad0', array([[492.57142857]])), <br>('random_guy0', array([[479.42857143]])), <br>('always_cooperate0', array([[474.28571429]])), <br>('tit_for_tat0', array([[450.07142857]])), <br>('smart_guy0', array([[443.71428571]])), <br>('always_defect0', array([[364.28571429]]))]
<br>New Population: ['mainly_nice', 'mainly_nice', 'always_cooperate', 'always_defect', 'mainly_nice', 'mainly_bad', 'tit_for_tat', 'random_guy', 'smart_guy']
Top strategy in iteration 1: mainly_nice

Iteration 2:
[('random_guy0', array([[540.66666667]])), <br>('mainly_nice0', array([[529.61111111]])), <br>('mainly_nice1', array([[511.94444444]])), <br>('smart_guy0', array([[504.27777778]])), <br>('mainly_nice2', array([[503.83333333]])), <br>('always_cooperate0', array([[502.33333333]])), <br>('tit_for_tat0', array([[489.22222222]])), <br>('always_defect0', array([[482.77777778]])), <br>('mainly_bad0', array([[470.88888889]]))]
<br>New Population: ['random_guy', 'random_guy', 'random_guy', 'mainly_nice', 'mainly_nice', 'always_cooperate', 'always_defect', 'mainly_nice', 'mainly_bad', 'tit_for_tat', 'random_guy', 'smart_guy']
Top strategy in iteration 2: random_guy

Iteration 3:
[('random_guy1', array([[534.70833333]])), <br>('random_guy2', array([[521.41666667]])), <br>('random_guy3', array([[513.625]])), <br>('always_defect0', array([[511.45833333]])), <br>('mainly_nice1', array([[508.29166667]])), <br>('smart_guy0', array([[503.04166667]])), <br>('random_guy0', array([[502.5]])), <br>('mainly_bad0', array([[501.875]])), <br>('mainly_nice0', array([[488.20833333]])), <br>('mainly_nice2', array([[485.41666667]])), <br>('tit_for_tat0', array([[475.45833333]])), <br>('always_cooperate0', array([[468.75]]))]
<br>New Population: ['random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'mainly_nice', 'mainly_nice', 'always_cooperate', 'always_defect', 'mainly_nice', 'mainly_bad', 'tit_for_tat', 'random_guy', 'smart_guy']
Top strategy in iteration 3: random_guy

Iteration 4:
[('random_guy5', array([[529.96875]])), <br>('random_guy6', array([[520.5625]])), <br>('random_guy4', array([[517.46875]])), <br>('random_guy0', array([[515.03125]])), <br>('random_guy2', array([[501.5]])), <br>('random_guy3', array([[501.03125]])), <br>('mainly_nice1', array([[500.15625]])), <br>('mainly_bad0', array([[489.125]])), <br>('mainly_nice0', array([[487.09375]])), <br>('random_guy1', array([[486.8125]])), <br>('mainly_nice2', array([[480.46875]])), <br>('smart_guy0', array([[478.1875]])), <br>('tit_for_tat0', array([[475.625]])), <br>('always_defect0', array([[475.3125]])), <br>('random_guy7', array([[468.3125]])), <br>('always_cooperate0', array([[452.1875]]))]
<br>New Population: ['random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'mainly_nice', 'mainly_nice', 'always_cooperate', 'always_defect', 'mainly_nice', 'mainly_bad', 'tit_for_tat', 'random_guy', 'smart_guy']
Top strategy in iteration 4: random_guy

Iteration 5:
[('random_guy7', array([[514.07142857]])), <br>('random_guy12', array([[511.5]])), <br>('random_guy4', array([[511.26190476]])), <br>('always_defect0', array([[510.]])), <br>('random_guy1', array([[502.92857143]])), <br>('random_guy2', array([[502.80952381]])), <br>('mainly_bad0', array([[500.19047619]])), <br>('random_guy6', array([[499.66666667]])), <br>('random_guy3', array([[498.04761905]])), <br>('random_guy8', array([[492.95238095]])), <br>('random_guy11', array([[490.69047619]])), <br>('random_guy10', array([[489.45238095]])), <br>('mainly_nice0', array([[488.16666667]])), <br>('mainly_nice1', array([[487.38095238]])), <br>('mainly_nice2', array([[486.5952381]])), <br>('random_guy9', array([[482.73809524]])), <br>('tit_for_tat0', array([[482.73809524]])), <br>('random_guy5', array([[479.66666667]])), <br>('smart_guy0', array([[470.52380952]])), <br>('random_guy0', array([[468.69047619]])), <br>('always_cooperate0', array([[454.19047619]]))]
<br>New Population: ['random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'random_guy', 'mainly_nice', 'mainly_nice', 'always_cooperate', 'always_defect', 'mainly_nice', 'mainly_bad', 'tit_for_tat', 'random_guy', 'smart_guy']
Top strategy in iteration 5: random_guyn iteration 5: random_guyiteration 5: random_guy